In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import re
movies_df = pd.read_csv('../data/movies.csv')
ratings_df = pd.read_csv('../data/ratings.csv')

In [67]:
#creamos una columna nueva que se llamará year
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Quitando los paréntesis 
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Eliminar los años de la columna "título" y el espacio
for i in range(len(movies_df)):
    movies_df['title'][i] = re.sub(r"\(\d{4}\)","",movies_df['title'][i])
    # print(i)
movies_df
movies_df = movies_df.drop('genres',axis=1)
movies_df

/tmp/ipykernel_35023/3479897439.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['title'][i] = re.sub(r"\(\d{4}\)","",movies_df['title'][i])
/tmp/ipykernel_35023/3479897439.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['title'][i] = re.sub(r"\(\d{4}\)","",movies_df['title'][i])
/tmp/ipykernel_35023/3479897439.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['title'][i] = re.sub(r"\(\d{4}\)","",

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic,2017
9738,193583,No Game No Life: Zero,2017
9739,193585,Flint,2017
9740,193587,Bungo Stray Dogs: Dead Apple,2018


In [68]:
userInput = [
{'title':'Heat', 'rating':5},
{'title':'Star Wars: Episode II - Attack of the Clones ', 'rating':4.5},
{'title':'Secret in Their Eyes, The (El secreto de sus ojos) ', 'rating':4.5},
{'title':'Terminator 2: Judgment Day ', 'rating':4.5},
{'title':'Toy Story ', 'rating':5},
{'title':'Halloween II ', 'rating':4.5}
]
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,Heat,5.0
1,Star Wars: Episode II - Attack of the Clones,4.5
2,"Secret in Their Eyes, The (El secreto de sus o...",4.5
3,Terminator 2: Judgment Day,4.5
4,Toy Story,5.0
5,Halloween II,4.5


In [69]:
#Filtrar las películas por título 
inputId = movies_df[movies_df["title"].isin(inputMovies.title)]
#Luego fusionándolo para que podamos obtener el movieId. Lo está fusionando implícitamente por título.
inputMovies = pd.merge(inputId, inputMovies)
#Eliminar información que no usaremos del dataframe
inputMovies = inputMovies.drop('year', axis=1)
inputMovies


,movieId,title,rating
0,1,Toy Story,5.0
1,589,Terminator 2: Judgment Day,4.5
2,1983,Halloween II,4.5
3,70994,Halloween II,4.5
4,5378,Star Wars: Episode II - Attack of the Clones,4.5
5,71033,"Secret in Their Eyes, The (El secreto de sus o...",4.5


In [70]:
#Filtrar a los usuarios que han visto películas que la entrada ha visto y almacenarlas
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'])]
# userSubset[userSubset['movieId']==1]
userSubset

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
516,5,1,4.0,847434962
552,5,589,3.0,847435258
806,6,589,3.0,845553317
874,7,1,4.5,1106635946
...,...,...,...,...
99519,609,589,3.0,847220990
99534,610,1,5.0,1479542900
99573,610,589,5.0,1479542983
99902,610,5378,3.5,1479544549


In [74]:
# Groupby crea varios sub dataframes de datos donde todos tienen el mismo valor en la columna especificada #como parámetro 
userSubsetGroup = userSubset.groupby(['userId'])
# Ordenamos por los usuarios que han puntuado mas a las peliculas en común con la entrada
userSubsetGroup = sorted(userSubsetGroup, key=lambda x: len(x[1]), reverse=True)
userSubsetGroup = userSubsetGroup[0:50]
userSubsetGroup

[((380,),
         userId  movieId  rating   timestamp
  56878     380        1     5.0  1493420345
  56958     380      589     5.0  1493473853
  57129     380     1983     3.0  1536874792
  57438     380     5378     3.0  1494528183
  57834     380    70994     1.0  1493421695),
 ((274,),
         userId  movieId  rating   timestamp
  39229     274        1     4.0  1171410158
  39339     274      589     4.5  1171409970
  39977     274     5378     2.5  1171409778
  40514     274    70994     3.5  1261091214),
 ((288,),
         userId  movieId  rating   timestamp
  42114     288        1     4.5  1054568869
  42191     288      589     4.0   976139234
  42442     288     1983     2.0   976139017
  42954     288     5378     3.5  1054568537),
 ((448,),
         userId  movieId  rating   timestamp
  68655     448        1     5.0  1019126661
  68747     448      589     3.0  1019124194
  69443     448     5378     3.0  1119469511
  69825     448    71033     4.0  1324744291),
 ((474,

In [75]:
pearsonCorrelationDict = {}
import math
#Para cada grupo de usuarios de nuestro subconjunto 
#Guardar la Correlación Pearson en un diccionario, donde la clave es el Id del usuario y el valor es el coeficiente

pearsonCorrelationDict = {}

 

#Para cada grupo de usuarios en nuestro subconjunto 

for name, group in userSubsetGroup:

    #Comencemos ordenando el usuario actual y el ingresado de forma tal que los valores no se mezclen luego

    group = group.sort_values(by='movieId')

    inputMovies = inputMovies.sort_values(by='movieId')

    #Obtener el N para la fórmula

    nRatings = len(group)

    #Obtener los puntajes de revisión para las películas en común

    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]

    #Guardarlas en una variable temporal con formato de lista para facilitar cálculos futuros

    tempRatingList = temp_df['rating'].tolist()

    #Pongamos también las revisiones de grupos de usuarios en una lista

    tempGroupList = group['rating'].tolist()

    #Calculemos la Correlación Pearson entre dos usuarios, x e y18
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)

    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)

    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)

 

    #Si el denominador es diferente a cero, entonces dividir, sino, la correlación es 0.

    if Sxx != 0 and Syy != 0:

        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)

    else:

        pearsonCorrelationDict[name] = 0


In [76]:
pearsonCorrelationDict.items()

dict_items([((380,), 0.534522483824843), ((274,), 0.29277002188455997), ((288,), 0.6172133998483676), ((448,), 0.8703882797784892), ((474,), 0.5222329678670935), ((610,), 0.47140452079103173), ((7,), 0.8660254037844264), ((21,), -0.5000000000000107), ((45,), -0.999999999999968), ((63,), 0.8029550685469584), ((68,), -0.8660254037844264), ((91,), 0), ((105,), 0), ((182,), 0.8660254037844264), ((186,), -0.4999999999999867), ((219,), 0.11470786693528257), ((220,), 0.5000000000000053), ((226,), -0.5000000000000107), ((232,), -0.9999999999999787), ((239,), 0.9999999999999787), ((247,), 0.8660254037844264), ((249,), 0.5000000000000107), ((279,), 0.18898223650460821), ((292,), 0.4999999999999867), ((298,), -0.8660254037844264), ((330,), 0.5000000000000107), ((332,), 0.6933752452815223), ((334,), 0.0), ((381,), 0.49999999999998535), ((414,), -0.4999999999999867), ((432,), -0.9449111825230502), ((434,), 0.38124642583150403), ((438,), 0.18898223650461649), ((453,), 0.5000000000000053), ((477,), -

In [77]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()


,similarityIndex,userId
0,0.534522,"(380,)"
1,0.292770,"(274,)"
2,0.617213,"(288,)"
3,0.870388,"(448,)"
4,0.522233,"(474,)"


In [80]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
48,1.000000,"(5,)"
19,1.000000,"(239,)"
3,0.870388,"(448,)"
13,0.866025,"(182,)"
6,0.866025,"(7,)"


In [81]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()


In [ ]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

In [ ]:
#Se crea un dataframe vacío
recommendation_df = pd.DataFrame()
#Ahora se toma el promedio ponderado
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

In [ ]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

In [ ]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]